In [3]:
import os
import glob
import keras
from keras_video import VideoFrameGenerator
import keras_video.utils
import tensorflow
from tensorflow.keras import backend
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import (Conv2D, MaxPooling3D, Conv3D,MaxPooling2D)
from tensorflow.keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from collections import deque
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization
from keras.utils.vis_utils import plot_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
import os.path
import time


# Loading the Model From the saved JSON File

In [4]:
# Set the path to the saved location of the JSON file. 

with open('F:/DLProj/model.json', 'r') as json_file:
    json_savedModel= json_file.read()
#load the model architecture 
model_j = tensorflow.keras.models.model_from_json(json_savedModel)
model_j.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_84 (TimeDis (None, 15, 98, 98, 32)    4736      
_________________________________________________________________
time_distributed_85 (TimeDis (None, 15, 96, 96, 32)    9248      
_________________________________________________________________
time_distributed_86 (TimeDis (None, 15, 96, 96, 32)    128       
_________________________________________________________________
time_distributed_87 (TimeDis (None, 15, 96, 96, 32)    0         
_________________________________________________________________
time_distributed_88 (TimeDis (None, 15, 48, 48, 32)    0         
_________________________________________________________________
time_distributed_89 (TimeDis (None, 15, 48, 48, 64)    18496     
_________________________________________________________________
time_distributed_90 (TimeDis (None, 15, 48, 48, 64)   

In [52]:
# Load the saved model weight
model_j.load_weights('F:/DLProj/Weights/model_weights.hdf5')

# Defining the video generator for the test set

In [53]:
import os
import glob
import keras
from keras_video import VideoFrameGenerator

# use sub directories names as classes
classes = [i.split(os.path.sep)[1] for i in glob.glob('F:/DLProj/1_2_Sub/Dataset/Comp/Test/*')]
classes.sort()
# some global params
SIZE = (300, 300)
CHANNELS = 3
NBFRAME = 15
BS = 5
# pattern to get videos and classes
glob_pattern='F:/DLProj/1_2_Sub/Dataset/Comp/Test/{classname}/*.mp4'
# for data augmentation
data_aug = keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2)
# Create video frame generator
test = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=0.00001,  # 33% of the videos will be used as validation set. 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_aug,
    use_frame_cache=False)

class Door, validation count: 0, train count: 22
class Not_Door, validation count: 0, train count: 15
Total data: 2 classes for 37 files for train


In [54]:
optimizer= RMSprop(lr=1e-4)
model_j.compile(optimizer=optimizer , 
              loss='categorical_crossentropy',
              #loss='binary_crossentropy',
              metrics=['accuracy'])

In [55]:
histos = model_j.evaluate_generator(test,
                                    steps = 10,
                                   verbose = 1
                                   )

  ...
    to  
  ['...']
10/10 [==============================] - 27s 3s/step - loss: 1.0827 - accuracy: 0.8000


In [56]:
print('Test score:', histos[0])
print('Test accuracy:', histos[1])

Test score: 1.0827441096305848
Test accuracy: 0.8
